In [ ]:
import numpy as np
import os
import glob
import matplotlib.cm as cm
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import sys

sys.path.append('.../Dropbox/bats-code')

from bat_functions import draw_tracks_on_frame

In [ ]:
def is_in_padding(position, padding, frame_width):
    """ Check if the point is in the part of the frame
        that the NN doesn't see because no padding used.
        
    position: (x, y)
    padding: int,  how much the image edge isn't seen
    """
    
    if position[0] < padding:
        return True
    elif position[0] > frame_width - padding:
        return True
    else:
        return False

def check_track_crossing(track, frame_num, line_height, frame_width, padding):
    """ Check is a given bat is crossing line in given frame.
    Return -1 if coming back, 1 is leaving
    
    Track: track object
    frame_num: frame to investigate
    line_height: what counts as crossing
    frame_width: width of frame
    padding: how much of the border of image does NN method miss
    """
    if track['first_frame'] > frame_num:
        # Not in frame
        return 0
    if track['first_frame'] + track['track'].shape[0] <= frame_num:
        # track already gone
        return 0
    
    rel_frame_num = frame_num - track['first_frame']
    next_frame = rel_frame_num + 1
    
    if next_frame >= track['track'].shape[0]:
        # Track ends on this frame
        return 0 

    if track['track'][rel_frame_num, 1] <= line_height:
        # Could be coming back
        if track['track'][next_frame, 1] > line_height:
            # Is coming back
            if is_in_padding(track['track'][next_frame], 
                             padding, frame_width):
                return 0
            return -1 
    if track['track'][rel_frame_num, 1] >= line_height:
        # Could be leaving
        if track['track'][next_frame, 1] < line_height:
            # Is leaving
            if is_in_padding(track['track'][next_frame], 
                             padding, frame_width):
                return 0
            return 1
    else:
        return 0
    
    
def count_bats_in_clip(clip_length, tracks, first_frame, line_height, frame_width, padding):
    
    num_coming = 0
    num_going = 0
    
    tracks = sorted(tracks, key=lambda t: t['first_frame'])
    
    for frame_num in range(clip_length):
        for track in tracks:
            if track['first_frame'] > first_frame + frame_num:
                break
            if track['last_frame'] < first_frame + frame_num:
                continue
            crossing = check_track_crossing(track, 
                                            first_frame+frame_num, 
                                            line_height,
                                            frame_width,
                                            padding
                                           )
            if crossing == -1:
                num_coming += 1
            if crossing == 1:
                num_going += 1
                
    return num_coming, num_going

In [ ]:

root_tracks_folder = '.../kasanka-bats/processed'
root_nn_tracks_folder = '.../kasanka-bats/processed/deep-learning'
frame_root_folder = '.../Elements/bats'
validation_csv = pd.read_csv('.../bats-data/bat_counting_error_quantification-Sheet1-raw.csv')





In [ ]:
cameras_frame_shift = {'16Nov':{'Chyniangale': 0,
               'BBC': -1209,
               'FibwePublic': 0,
               'MusoleTower': 0,
               'Puku': 0,
               'FibweParking': 0,
               'MusoleParking': 0,
               'MusolePath': 0,
               'Sunset': 0,
               'NotChyniangale': 0,
               },
               '17Nov':{'Chyniangale': 0,
               'BBC': 0,
               'FibwePublic': 0,
               'MusoleTower': 0,
               'Puku': 0,
               'FibweParking2': -805,
               'MusoleParking': 0,
               'MusolePath2': 0,
               'Sunset': 0,
               'NotChyniangale': 0,
               }
              }

In [ ]:
# camera_name = 'NotChyniangale'

# date = '17Nov'

# frame_files = sorted(
#                 glob.glob(os.path.join(frame_root_folder, 
#                                        date,
#                                        camera_name, 
#                                        "*", 
#                                        "*.jpg")
#                          )
#             )

# positions_file = os.path.join(root_nn_tracks_folder, date, camera_name, 'centers.npy')
# positions = np.load(positions_file, allow_pickle=True)

# frame_shift = len(positions) - len(frame_files)
# frame_shift = 0


# frame_num = 30000
# shift = 48

# frame = plt.imread(frame_files[frame_num])
# plt.figure(figsize=(20,20))
# plt.imshow(frame)
# plt.scatter(positions[frame_num+frame_shift][:, 0]+shift, 
#             positions[frame_num+frame_shift][:, 1]+shift, 
#             s=1, c='r')
# plt.title(frame_shift)

In [ ]:
# len(positions),  len(frame_files)

In [ ]:
# clip_files = glob.glob(os.path.join(root_output_folder, '*', '*.mp4'))
# clip_files = sorted(clip_files, key=lambda f: (f.split('-')[-6], int(f.split('-')[-2])))

In [ ]:
# validation_csv.loc[0]

In [ ]:
# clip_files_with_ext = validation_csv['Video clip name'].values
# clip_files = []
# for clip_file in clip_files_with_ext:
#     name, ext = os.path.splitext(clip_file)
#     clip_files.append(name)

In [ ]:
# for t in tracks:
#     next_frame = np.min([2+1, t['track'].shape[0]-1])

In [ ]:
# for t in tracks:
#     next_frame = t['first_frame'] + 1
#     if next_frame < t['track'].shape[0]:
#         next_frame = 0   

In [ ]:
falloff_mode = False


shift = 0
camera_num = -1
frame_shift = 0

frames_camera = None
# positions_camera
frames = []

last_track_file = None

last_positions_file = None

for index, row in validation_csv.iterrows():
    
    
    
    if (index+3) % 6 == 0:
        print(f'camera num: {camera_num}')
        if falloff_mode:
            clip_file = row['video-clip-name']
        else:
            clip_file = row['Video clip name']
        name, ext = os.path.splitext(clip_file)
        if falloff_mode:
            first_frame = int(name.split('-')[-4])
            camera_name = name.split('-')[-8]
        else:
            first_frame = int(name.split('-')[-2])
            camera_name = name.split('-')[-6]
        print(camera_name, first_frame)
        if camera_name != frames_camera:
            camera_num += 1
            if falloff_mode:
                date = row['date-folder']
            else:
                date = row['date_folder']
            date = date.replace(' ', '')
            frame_files = sorted(
                glob.glob(os.path.join(frame_root_folder, 
                                       date,
                                       camera_name, 
                                       "*", 
                                       "*.jpg")
                         )
            )
            tracks_folder = os.path.join(root_nn_tracks_folder, date)
            track_file = os.path.join(tracks_folder, camera_name, 'raw_tracks.npy')
            print(track_file)
            if track_file != last_track_file:
                print('loading tracks...')
                tracks = np.load(track_file, allow_pickle=True)
                last_track_file = track_file
            frames_camera = camera_name
#             positions_file = os.path.join(root_nn_tracks_folder, date, 
#                                           camera_name, 'centers.npy')
#             if positions_file != last_positions_file:
#                 positions = np.load(positions_file, allow_pickle=True)
#                 last_positions_file = positions_file
            
            frame_shift = cameras_frame_shift[date][camera_name]
            
#         if camera_name == 'BBC':

#         plt.figure()
        frame = plt.imread(frame_files[first_frame])
#         plt.imshow(frame)
        if first_frame+frame_shift >= 0:
            draw_tracks_on_frame(frame, first_frame+frame_shift, 
                                 tracks, figure_scale=100, shift=shift)
            print(str(first_frame) in frame_files[first_frame], frame_files[first_frame])
        else:
            print('Shift is negative.')
    

## Add darkness info

In [ ]:
if falloff_mode:
    validation_csv['darkness-mean'] = None
    validation_csv['darkness-max'] = None
    validation_csv['darkness-median'] = None
else:
    validation_csv['darkness_mean'] = None
    validation_csv['darkness_max'] = None
    validation_csv['darkness_median'] = None

for index, row in validation_csv.iterrows():
    if falloff_mode:
        clip_file = row['video-clip-name']
    else:
        clip_file = row['Video clip name']
    name, ext = os.path.splitext(clip_file)
    if falloff_mode:
        first_frame = int(name.split('-')[-4])
        camera_name = name.split('-')[-8]
    else:
        first_frame = int(name.split('-')[-2])
        camera_name = name.split('-')[-6]

    if camera_name != frames_camera:
        camera_num += 1
        if falloff_mode:
            date = row['date-folder']
        else:
            date = row['date_folder']
        date = date.replace(' ', '')
        frame_files = sorted(
            glob.glob(os.path.join(frame_root_folder, 
                                   date,
                                   camera_name, 
                                   "*", 
                                   "*.jpg")
                     )
        )

        frames_camera = camera_name
        frame_shift = cameras_frame_shift[date][camera_name]

    frame = plt.imread(frame_files[first_frame])
    blue_ind = 2
    if falloff_mode:
        validation_csv.loc[index, 'darkness-mean'] = np.mean(frame[..., blue_ind])
        validation_csv.loc[index, 'darkness-max'] = np.max(frame[..., blue_ind])
        validation_csv.loc[index, 'darkness-median'] = np.median(frame[..., blue_ind])
    else:
        validation_csv.loc[index, 'darkness_mean'] = np.mean(frame[..., blue_ind])
        validation_csv.loc[index, 'darkness_max'] = np.max(frame[..., blue_ind])
        validation_csv.loc[index, 'darkness_median'] = np.median(frame[..., blue_ind])

In [ ]:
# ind = validation_csv['Video clip name']=='test-camera-MusoleTower-clip-4-firstframe-52685-5'
# validation_csv.loc[ind, 'total_bats'] = 807

In [ ]:
validation_csv.head()

In [ ]:
if falloff_mode:
    validation_csv['old-method-count-coming'] = None
    validation_csv['old-method-count-going'] = None
    validation_csv['new-method-count-coming'] = None
    validation_csv['new-method-count-going'] = None
else:
    validation_csv['old_method_count_coming'] = None
    validation_csv['old_method_count_going'] = None
    validation_csv['new_method_count_coming'] = None
    validation_csv['new_method_count_going'] = None

In [ ]:

if falloff_mode:
    clip_time = 7
else:
    clip_time = 15 # in seconds (30fps)

line_height_fraction = .5

frame_files = glob.glob(os.path.join(frame_root_folder, "16Nov", "BBC", "*", "*.jpg"))
frame_size = cv2.imread(frame_files[-1]).shape
line_height = frame_size[0] * line_height_fraction
frame_width = frame_size[1]

last_camera_name = None

for new_method in [True, False]:
    for index, row in validation_csv.iterrows():
        if falloff_mode:
            clip_file = row['video-clip-name']
        else:
            clip_file = row['Video clip name']
        name, ext = os.path.splitext(clip_file)
        if falloff_mode:
            first_frame = int(name.split('-')[-4])
            camera_name = name.split('-')[-8]
        else:
            first_frame = int(name.split('-')[-2])
            camera_name = name.split('-')[-6]

        if camera_name != last_camera_name:
            print('loading...')
            if falloff_mode:
                date = row['date-folder']
            else:
                date = row['date_folder']
            date = date.replace(' ', '')
            if new_method:
                tracks_folder = os.path.join(root_nn_tracks_folder, date)
            else:
                tracks_folder = os.path.join(root_tracks_folder, date)
            track_file = os.path.join(tracks_folder, camera_name, 'raw_tracks.npy')
            print(track_file)
            tracks = np.load(track_file, allow_pickle=True)

        clip_length = int(30 * clip_time)
        if new_method:
            frame_shift = cameras_frame_shift[date][camera_name]
        else:
            frame_shift = 0
        num_coming, num_going = count_bats_in_clip(
            clip_length, tracks, first_frame+frame_shift, line_height, frame_width, padding=48)
#         print(f'{camera_name} {first_frame+frame_shift}, coming: {num_coming}, going: {num_going}')
        if falloff_mode:
            if new_method:
                validation_csv.loc[index, 'new-method-count-coming'] = num_coming 
                validation_csv.loc[index, 'new-method-count-going'] = num_going
            else:
                validation_csv.loc[index, 'old-method-count-coming'] = num_coming
                validation_csv.loc[index, 'old-method-count-going'] = num_going
        else:
            if new_method:
                validation_csv.loc[index, 'new_method_count_coming'] = num_coming 
                validation_csv.loc[index, 'new_method_count_going'] = num_going
            else:
                validation_csv.loc[index, 'old_method_count_coming'] = num_coming
                validation_csv.loc[index, 'old_method_count_going'] = num_going

        last_camera_name = camera_name

    #     if index == 3:
    #         break

In [ ]:
frame_width

In [ ]:
validation_csv.head()

In [ ]:
validation_csv.to_csv('.../bats-data/bat_counting_error_quantification_with_counts_padded.csv')
# validation_csv.to_csv(save_file)

In [ ]:
validation_csv = '.../bats-data/bat_counting_error_quantification_with_counts_padded.csv'
validation_csv = pd.read_csv(validation_csv)

In [ ]:
validation_csv.head()

In [ ]:
if falloff_mode:
    validation_csv['total-bats'] = (
        (validation_csv['new-method-count-going'] 
         - pd.to_numeric(validation_csv['number-false-detections-going(blue)'], errors='coerce')
         + pd.to_numeric(validation_csv['number-missed-detections-going'], errors='coerce')
        )
        - (validation_csv['new-method-count-coming'] 
           - pd.to_numeric(validation_csv['number-false-detections-coming(yellow)'], errors='coerce')
           + pd.to_numeric(validation_csv['number-missed-detections-coming'], errors='coerce')
          )
    )
else:
    validation_csv['total_bats'] = (
        (validation_csv['old_method_count_going'] - pd.to_numeric(validation_csv['Number false detections moving up (blue)'], errors='coerce'))
        - (validation_csv['old_method_count_coming'] - pd.to_numeric(validation_csv['Number false detections moving down (yellow)'], errors='coerce'))
        + pd.to_numeric(validation_csv['Number missed detections'], errors='coerce')
    )

In [ ]:
if falloff_mode:
    validation_csv['total-bats-old-method'] = (validation_csv['old-method-count-going'] 
                                               - validation_csv['old-method-count-coming']
                                              )

    validation_csv['total-bats-new-method'] = (validation_csv['new-method-count-going'] 
                                               - validation_csv['new-method-count-coming']
                                              )
else:
    validation_csv['total_bats_old_method'] = (validation_csv['old_method_count_going'] 
                                               - validation_csv['old_method_count_coming']
                                              )

    validation_csv['total_bats_new_method'] = (validation_csv['new_method_count_going'] 
                                               - validation_csv['new_method_count_coming']
                                              )

In [ ]:
if falloff_mode:
    not_zero = validation_csv['total-bats'] != 0
    validation_csv['old-method-fraction-total'] = np.nan
    validation_csv['new-method-fraction-total'] = np.nan
    validation_csv.loc[not_zero, 'old-method-fraction-total'] = (
        validation_csv['total-bats-old-method'][not_zero] / validation_csv['total-bats'][not_zero]
    )
    validation_csv.loc[not_zero, 'new-method-fraction-total'] = (
        validation_csv['total-bats-new-method'][not_zero] / validation_csv['total-bats'][not_zero]
    )
else:
    not_zero = validation_csv['total_bats'] != 0
    validation_csv['old_method_fraction_total'] = np.nan
    validation_csv['new_method_fraction_total'] = np.nan
    validation_csv.loc[not_zero, 'old_method_fraction_total'] = validation_csv['total_bats_old_method'][not_zero] / validation_csv['total_bats'][not_zero]
    validation_csv.loc[not_zero, 'new_method_fraction_total'] = validation_csv['total_bats_new_method'][not_zero] / validation_csv['total_bats'][not_zero]

In [ ]:
if falloff_mode:
    print(validation_csv['old-method-fraction-total'].mean())
    print(validation_csv['new-method-fraction-total'].mean())
else:
    print(validation_csv['old_method_fraction_total'].mean())
    print(validation_csv['new_method_fraction_total'].mean())

In [ ]:
print(validation_csv['old_method_fraction_total'].mean())
print(validation_csv['new_method_fraction_total'].mean())

In [ ]:
min_num = 500
print(validation_csv.loc[validation_csv['total_bats']>min_num,'old_method_fraction_total'].mean())
print(validation_csv.loc[validation_csv['total_bats']>min_num,'new_method_fraction_total'].mean())

In [ ]:
print(validation_csv.loc[~validation_csv['total_bats'].isnull(), 'total_bats_old_method'].sum() 
      /  validation_csv['total_bats'].sum())
print(validation_csv['total_bats_old_method'].sum() 
      /  validation_csv['total_bats'].sum())

print((validation_csv.loc[~validation_csv['total_bats'].isnull(), 'total_bats_new_method'].sum()) 
      /  (validation_csv['total_bats'].sum()))
print((validation_csv['total_bats_new_method'].sum()) 
      /  (validation_csv['total_bats'].sum()))

print(validation_csv.loc[validation_csv['total_bats'].isnull(), 'total_bats_new_method'].sum())
print(validation_csv.loc[validation_csv['total_bats'].isnull(), 'total_bats_old_method'].sum())

In [ ]:
~validation_csv['total_bats_new_method'].isnull()

In [ ]:
min(validation_csv['total_bats'])

In [ ]:
bin_size = 100


plt.scatter((validation_csv['total_bats']//bin_size)*bin_size-2, 
            validation_csv['total_bats'] - validation_csv['total_bats_new_method'],
            s=1, c='r')


plt.scatter((validation_csv['total_bats']//bin_size)*bin_size+2, 
            validation_csv['total_bats'] - validation_csv['total_bats_old_method'], 
            s=1)

plt.figure()

plt.scatter((validation_csv['total_bats']//bin_size)*bin_size-2, 
            validation_csv['new_method_fraction_total'],
            s=1, c='r')


# plt.scatter((validation_csv['total_bats']//bin_size)*bin_size+2, 
#             validation_csv['old_method_fraction_total'], 
#             s=1)
plt.figure()

plt.scatter(validation_csv['total_bats'], 
            validation_csv['total_bats_new_method'],
            s=1, c='r')
plt.gca().set_aspect('equal')
plt.plot([0,1500], [0,1500])

In [ ]:
# validation_csv.groupby('bin_50')['abs_error_new'].mean()

In [ ]:
validation_csv['abs_error_old'] = validation_csv['total_bats'] - validation_csv['total_bats_old_method']
validation_csv['abs_error_new'] = validation_csv['total_bats'] - validation_csv['total_bats_new_method']

In [ ]:
plt.scatter(validation_csv['bin_50']-2, 
            validation_csv['abs_error_new'],
            s=1, c='r', alpha=.5)


plt.scatter(validation_csv['bin_50']+2, 
            validation_csv['abs_error_old'], 
            s=1, alpha=.5)

# validation_csv.groupby('bin_50').mean()

plt.figure()

plt.scatter(validation_csv['bin_50']-2, 
            validation_csv['new_method_fraction_total'],
            s=1, c='r', alpha=.5)


plt.scatter(validation_csv['bin_50']+2, 
            validation_csv['old_method_fraction_total'], 
            s=1, alpha=.5)

plt.figure()

plt.scatter(validation_csv['bin_50'], 
            validation_csv['total_bats'],
            s=1, c='r', alpha=.5)


# plt.scatter(validation_csv['bin_50']+2, 
#             validation_csv['old_method_fraction_total'], 
#             s=1, alpha=.5)

In [ ]:
np.max((validation_csv['darkness_max']//50)*50+2)

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.violinplot(x="day", y="total_bill", hue="smoker",
                    data=tips, palette="muted")

In [ ]:
validation_csv.loc[validation_csv['new_method_fraction_total'] < .4]

In [ ]:
per_list = []
total_list = []
miss_list = []
for ind in range(len(validation_csv)):
    alg_count = validation_csv.loc[ind]['new_method_count_coming'] + validation_csv.loc[ind]['new_method_count_going']
    missed = validation_csv.loc[ind]['Number missed detections']
    try:
        missed = int(missed)
    except:
        continue
    total = alg_count + missed
    if total != 0:
        percent_seen = alg_count / total
        per_list.append(percent_seen)
        total_list.append(total)
        miss_list.append(missed)
#         print(percent_seen, alg_count, missed)

In [ ]:
for p, t, m in zip(per_list, total_list, miss_list):
    if p < .4:
        print(p, t, m)
np.mean(np.array(per_list)), np.median(np.array(per_list))

In [ ]:
plt.hist(per_list, bins=100)

In [ ]:
plt.hist(per_list, bins=100)

In [ ]:
frame_files = glob.glob(os.path.join(frame_root_folder, camera_name, "*", "*.jpg"))

In [ ]:
files = glob.glob(os.path.join(frame_root_folder, '*'))

In [ ]:
for f in files:
    frame_files = glob.glob(os.path.join(f, "*", "*.jpg"))
    print(cv2.imread(frame_files[-1]).shape)